## 추가 사례 1: 불량품, 질병 판정의 자동화 (이진 분류, 재현율)

### 공통 전처리

In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

In [ ]:
# 혼동행렬 출력용 함수

def make_cm(matrix, columns):
    # matrix : 넘파이 배열
    # columns : 필드명 리스트
    n = len(columns)
    
    # '정답 데이터'를 n번 반복해 연접한 리스트
    act = ['정답데이터'] * n
    pred = ['예측결과'] * n
    
    # 데이터프레임 생성
    cm = pd.DataFrame(matrix, 
        columns=[pred, columns], index=[act, columns])
    return cm

### A1.4 데이터 읽어들이기부터 데이터 확인까지

### 필드 정보

#### age
나이

#### sex
성별
* 1: 남성
* 0: 여성

#### cp
흉통 종류
* 1：전형적인 협심증
* 2：비정형 협심증
* 3：비협심성 흉통
* 4：무증후성 

#### trestbps
안정시 혈압
（입원 시점, 단위: mmHg）
    
#### chol
혈장 콜레스테롤
（mg / dl）

#### fbs
공복시 혈당   
(> 120 (mg / dl）)
* 1 참
* 0 거짓

#### restecg
안정시 심전도
 
* 0：정상
* 1：ST-T파 이상（T파 반전 / ST 상승 혹은 억제 > 0.05 mV）
* 2：좌심실 비대 가능성

#### thalach
최대 심박수

#### exang
운동 유발성 협심증
* 1  예
* 0  아니오
    
#### oldpeak
ST 저하
(안정시와 비교해 운동에 의해 유발되는 ST 저하)

#### slope
ST 세그먼트 기울기
(피크 운동 ST 세그먼트 기울기)
* 1 상승
* 2 평탄
* 3 하강

#### ca 
주요 혈관 수
(X선 투시촬영에서 착색된 주요 혈관 수 (0~3))


#### thal
탈륨 
(탈륨 심장 스캔 결과)
* 3 : 정상
* 6 : 수정된 결함
* 7 : 회복 가능한 결함


#### num (목적변수)
심장병 여부
(혈관 조영 결과)
* 0: 소견 있음
* 1: 소견 없음

#### 데이터 읽어 들이기

In [ ]:
# 심장 질환 공개 데이터 집합 읽어 들이기

columns = [
    '나이', '성별', '흉통종류', '안정시혈압',  '혈장콜레스테롤',
    '공복시혈당', '안정시심전도',  '최대심박수',  '운동유발성협심증',
    'ST저하', 'ST세그먼트기울기', '주요혈관수', '탈륨', '심장병여부'
]

# 공개 데이터 집합 URL
url_hu = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'

# 데이터프레임으로 읽어 들이기
# 누락값은 '?'으로 표기되며, 읽어들인 후에는 파이썬의 NaN으로 변환됨
df_hu = pd.read_csv(url_hu, header=None, names=columns, na_values='?')

#### 데이터 확인

In [ ]:
# 데이터 전체 확인
display(df_hu.head())

In [ ]:
# 데이터 건수와 필드 수 확인
print(df_hu.shape)
print()

# '심장병여부' 필드의 값 분포 확인
print(df_hu['심장병여부'].value_counts())
print()

# 진단율 계산
rate = df_hu['심장병여부'].value_counts()[1]/len(df_hu)
print(f'진단율: {rate:.4f}')

In [ ]:
# 히스토그램 그리기

# 그래프 크기 조정
from pylab import rcParams
rcParams['figure.figsize'] = (12, 12)

# 데이터프레임의 수치 필드를 대상으로 한 히스토그램 그리기
df_hu.hist()
plt.show()

In [ ]:
# 누락값 확인
print(df_hu.isnull().sum())

### A1.5 데이터 전처리와 데이터 분할

#### 데이터 전처리

#### 누락값 처리 방법

* 안정시혈압: 평균
* 혈장콜레스테롤: 평균
* 공복시혈당: 0
* 안정시심전도: 0
* 최대심박수: 평균
* 운동유발성협심증: 0   
* ST세그먼트기울기: 2  
----    
* 주요혈관수: 필드 자체를 제거
* 탈륨: 필드 자체를 제거

In [ ]:
# 평균값 계산
ave1 = df_hu['안정시혈압'].mean()
ave2 = df_hu['혈장콜레스테롤'].mean()
ave3 = df_hu['최대심박수'].mean()

# 결과 확인
print(f'안정시혈압: {ave1:.1f}  혈장콜레스테롤: {ave2:.1f}   최대심박수:{ave3:.1f}')

In [ ]:
# 누락값 치환
df2 = df_hu.fillna(
    {'안정시혈압': ave1,
    '혈장콜레스테롤': ave2,
    '공복시혈당': 0,
    '안정시심전도': 0,
    '최대심박수': ave3,
    '운동유발성협심증': 0,
    'ST세그먼트기울기': 2}
)

# 필드 삭제
df3 = df2.drop(['주요혈관수', '탈륨'], axis=1)

In [ ]:
# 결과 확인
print(df3.isnull().sum())
display(df3.head())

#### 데이터 분할

In [ ]:
# 입력 데이터와 정답 데이터 분할
x = df3.drop('심장병여부', axis=1)
y = df3['심장병여부'].values

# 학습 데이터와 검증 데이터 분할
test_size=0.40

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
  x, y, test_size=test_size, random_state=random_seed,
  stratify=y)

### A1.6 알고리즘 선택하기

#### 알고리즘 선택

In [ ]:
# 후보 알고리즘의 리스트

# 로지스틱 회귀 (4.3.3)
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=random_seed)

# 결정 트리 (4.3.6)
from sklearn.tree import DecisionTreeClassifier
algorithm2 = DecisionTreeClassifier(random_state=random_seed)

# 랜덤 포레스트 (4.3.7)
from sklearn.ensemble import RandomForestClassifier
algorithm3 = RandomForestClassifier(random_state=random_seed)

# XGBoost (4.3.8)
from xgboost import XGBClassifier
algorithm4 = XGBClassifier(random_state=random_seed)

algorithms = [algorithm1, algorithm2, algorithm3, algorithm4]

In [ ]:
# 교차 검증법을 이용해 최적의 알고리즘을 선택

from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import cross_val_score
for algorithm in algorithms:
    # 교차 검증법
    scores = cross_val_score(algorithm , x_train, y_train,
        cv=stratifiedkfold, scoring='roc_auc')
    score = scores.mean()
    name = algorithm.__class__.__name__
    print(f'평균 점수: {score:.4f}  개별 점수: {scores}  {name}')

랜덤 포레스트의 성능이 가장 좋게 나왔으므로 랜덤 포레스트를 채택함



### A1.7 학습, 예측, 평가

In [ ]:
# 알고리즘 선택
# 랜덤 포레스트를 선택함
algorithm = RandomForestClassifier(random_state=random_seed)

# 학습
algorithm.fit(x_train, y_train)

# 예측
y_pred = algorithm.predict(x_test)

In [ ]:
# 평가

# 혼동행렬을 출력
from sklearn.metrics import confusion_matrix
df_matrix = make_cm(
    confusion_matrix(y_test, y_pred),
    ['소견없음', '소견있음'])
display(df_matrix)

# 정확률, 재현율, F-점수를 계산
from sklearn.metrics import precision_recall_fscore_support
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, 
    y_pred, average='binary')
print(f'정확률: {precision:.4f}  재현율: {recall:.4f}  F-점수: {fscore:.4f}')

### A1.8 튜닝

In [ ]:
# 확률의 도수분포 그래프

# y=1일 확률
y_proba1= algorithm.predict_proba(x_test)[:,1]

# y_test=0, y_test=1인 경우로 데이터를 분할
y0 = y_proba1[y_test==0]
y1 = y_proba1[y_test==1]

# 산포도 그리기
import seaborn as sns
plt.figure(figsize=(6,6))
plt.title('확률의 도수분포')
sns.distplot(y1, kde=False, norm_hist=True,
    bins=10,color='b', label='소견있음')
sns.distplot(y0, kde=False, norm_hist=True,
    bins=10,color='k', label='소견없음')
plt.xlabel('확률')
plt.legend()
plt.show()

#### 역치 수정하기

#### predict_proba 함수를 이용해 0.5외의 역치를 반영한 결과를 예측
(4.4절 참조)

In [ ]:
# 역치를 바꿔가며 예측 결과를 계산하는 함수
def pred(algorithm, x, thres):
    # 확률값(행렬)
    y_proba = algorithm.predict_proba(x)
    
    # 예측결과 1의 확률
    y_proba1 =  y_proba[:,1]
    
    # 예측결과 1일 확률이 역치보다 큰지 여부
    y_pred = (y_proba1 > thres).astype(int)
    return y_pred

In [ ]:
# 역치를 0.05단위로 바꿔가며 정확률, 재현율, F-점수를 계산
thres_list = np.arange(0.5, 0, -0.05)

for thres in thres_list:
    y_pred2 = pred(algorithm, x_test, thres)
    pred_sum =  y_pred2.sum()
    precision, recall, fscore, _ = precision_recall_fscore_support(
        y_test, y_pred2, average='binary')
    print(f'역치: {thres:.2f} 합계: {pred_sum}  정확률: {precision:.4f}\
  재현율: {recall:.4f}  F-점수: {fscore:.4f})')

In [ ]:
# 재현율을 중시해 역치를 0.25로 설정
y_final = pred(algorithm, x_test, 0.25)

# 혼동행렬을 출력
from sklearn.metrics import confusion_matrix
df_matrix4 = make_cm(
    confusion_matrix(y_test, y_final),
    ['소견없음', '소견있음'])
display(df_matrix4)

# 정확률, 재현율, F-점수를 계산
precision, recall, fscore, _ = precision_recall_fscore_support(
    y_test, y_final, average='binary')
print(f'정확률: {precision:.4f}  재현율: {recall:.4f}  F-점수: {fscore:.4f}')